                                                                            Luis Ramirez Camargo, June 2020

# Calculate accuracy indicators for PV output calculated with ERA5-land and MERRA 2 Data  

This notebook uses the outputs from the notebook clean_measured_pv_data_installations_chile to evaluate the accuracy of the outputs from  the notebook pv_output_from_ERA5_land_and_merra2. The accuracy is assessed using the pearsons coorelation coeficient, the Mean Biased Error (MBE), the root mean square error (rmse). 


## 1) import the libraries and data

In [1]:
import os
import itertools
import xarray as xr
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import glob
from tkinter import Tcl
import gc
import scipy as sp
import scipy.stats
import seaborn as sns
from pathlib import Path

## Import the capacity factors of the meassured data

In [2]:
reference = pd.read_csv(Path('intermediate_results/time_series_PV_sen_chile_capacity_factors.csv'),
                        index_col=0, parse_dates=True)
#errase the erroneous part of the time series of installations that despite erroneous shape
#still have more than one year of meassurements and can be taken into account for comparison
reference.loc[:'2017-09','SOLAR HORMIGA'] = np.nan
reference.loc[:'2017-11','SOLAR EL AGUILA I'] = np.nan

In [3]:
reference_optimal_des_rad = pd.read_csv(Path('intermediate_results/deseason_rad_optimal_reference.csv'),
                        index_col=0, parse_dates=True)
reference_tracking_des_rad = pd.read_csv(Path('intermediate_results/deseason_rad_tracking_reference.csv'),
                        index_col=0, parse_dates=True)
#errase the erroneous part of the time series of installations that despite erroneous shape
#still have more than one year of meassurements and can be taken into account for comparison
reference_optimal_des_rad.loc[:'2017-09','SOLAR HORMIGA'] = np.nan
reference_optimal_des_rad.loc[:'2017-11','SOLAR EL AGUILA I'] = np.nan

In [4]:
reference_non_tracking_cumulated = pd.read_csv(Path('intermediate_results/cumulated_non_tracking_reference.csv'),
                        index_col=0, parse_dates=True)
reference_tracking_cumulated = pd.read_csv(Path('intermediate_results/cumulated_tracking_reference.csv'),
                        index_col=0, parse_dates=True)

In [5]:
reference_non_tracking_des_cumulated = pd.read_csv(Path('intermediate_results/cumulated_deseason_non_tracking_reference.csv'),
                        index_col=0, parse_dates=True)
reference_tracking_des_cumulated = pd.read_csv(Path('intermediate_results/cumulated_deseason_tracking_reference.csv'),
                        index_col=0, parse_dates=True)

## 3) Import the capacity factors of the PV output calculated with ERA5-land data and MERRA2-data 

In [6]:
#modify the calculated data sets time series to match chilean summer time
#this cannot be performed automatically since the dailight summer time in chile changes from year to year
def summer_time_chile(cf_file):
    cf_utc = pd.read_csv(cf_file, index_col=0, parse_dates=True)
    cf_ut_st_2014 = cf_utc["2014-04-27":"2014-09-07"].shift(periods=-1).copy()
    cf_ut_st_2016 = cf_utc["2016-05-15":"2016-08-14"].shift(periods=-1).copy()
    cf_ut_st_2017 = cf_utc["2017-05-14":"2017-08-14"].shift(periods=-1).copy()
    cf_ut_st_2018 = cf_utc["2018-05-13":"2018-08-12"].shift(periods=-1).copy()
    cf_utc_st = cf_utc.copy()
    cf_utc_st.loc["2014-04-27":"2014-09-07"] = cf_ut_st_2014
    cf_utc_st.loc["2016-05-15":"2016-08-14"] = cf_ut_st_2016 
    cf_utc_st.loc["2017-05-14":"2017-08-14"] = cf_ut_st_2017
    cf_utc_st.loc["2018-05-13":"2018-08-12"] = cf_ut_st_2018
    return cf_utc_st

In [7]:
era5l_optimal = \
summer_time_chile(Path('intermediate_results/timeseries_capacity_factors_pv_optimal_era5l.csv'))
merra2_optimal = \
summer_time_chile(Path('intermediate_results/timeseries_capacity_factors_pv_optimal_merra2.csv'))
era5l_tracking = \
summer_time_chile(Path('intermediate_results/timeseries_capacity_factors_pv_tracking_era5l.csv'))
merra2_tracking = \
summer_time_chile(Path('intermediate_results/timeseries_capacity_factors_pv_tracking_merra2.csv'))

In [8]:
era5l_optimal_cumulated = \
summer_time_chile(Path('intermediate_results/cumulated_non_tracking_era5l.csv'))
merra2_optimal_cumulated = \
summer_time_chile(Path('intermediate_results/cumulated_non_tracking_merra2.csv'))
era5l_tracking_cumulated = \
summer_time_chile(Path('intermediate_results/cumulated_tracking_era5l.csv'))
merra2_tracking_cumulated = \
summer_time_chile(Path('intermediate_results/cumulated_tracking_merra2.csv'))

In [9]:
era5l_optimal_des = \
summer_time_chile(Path('intermediate_results/deseason_optimal_era5l.csv'))
merra2_optimal_des = \
summer_time_chile(Path('intermediate_results/deseason_optimal_merra2.csv'))
era5l_tracking_des = \
summer_time_chile(Path('intermediate_results/deseason_tracking_era5l.csv'))
merra2_tracking_des = \
summer_time_chile(Path('intermediate_results/deseason_tracking_merra2.csv'))

In [10]:
era5l_optimal_des_rad = \
summer_time_chile(Path('intermediate_results/deseason_rad_optimal_era5l.csv'))
merra2_optimal_des_rad = \
summer_time_chile(Path('intermediate_results/deseason_rad_optimal_merra2.csv'))
era5l_tracking_des_rad = \
summer_time_chile(Path('intermediate_results/deseason_rad_tracking_era5l.csv'))
merra2_tracking_des_rad = \
summer_time_chile(Path('intermediate_results/deseason_rad_tracking_merra2.csv'))

In [11]:
era5l_optimal_des_rad_cumulated = \
summer_time_chile(Path('intermediate_results/cumulated_deseason_non_tracking_era5l.csv'))
merra2_optimal_des_rad_cumulated = \
summer_time_chile(Path('intermediate_results/cumulated_deseason_non_tracking_merra2.csv'))
era5l_tracking_des_rad_cumulated = \
summer_time_chile(Path('intermediate_results/cumulated_deseason_tracking_era5l.csv'))
merra2_tracking_des_rad_cumulated = \
summer_time_chile(Path('intermediate_results/cumulated_deseason_tracking_merra2.csv'))

## 4) Indicators calculation for the validation ERA5-land

In [12]:
def validate_pv_output_all(reference_values_df,calculated_values_df, name_file_to_store):
    '''calculate the accuracy indicators for each plant where the reference data set is not empty'''
    indicators = pd.DataFrame(columns=("pearson_h",
                                       "mbe_h",
                                       "rmse_h",
                                       "observations",
                                       "pearson_d",
                                       "mbe_d",
                                       "rmse_d",
                                       "days",
                                       "pearson_m",
                                       "mbe_m",
                                       "rmse_m",
                                       "months"), index=calculated_values_df.columns)
    for plant in calculated_values_df.columns:
        if np.sum(reference_values_df[plant]["2014":"2018"]) != 0:
            #print(plant)
            comp_pre = pd.DataFrame(columns=("new","reference"), index=calculated_values_df.index)
            #print(np.array(calculated_values_df.index).size)
            comp_pre["new"] = np.array(calculated_values_df[plant]["2014":"2018"]).copy()
            #print(np.array(reference_values_df[plant]["2014":"2018"].copy()).size)
            #print(reference_values_df[plant]["2014-01-01"])
            comp_pre.loc[3:,"reference"] = reference_values_df[plant][:-5].values
            #print(comp_pre)
            comp = comp_pre.dropna(axis=0).copy()
            #print(comp)
            new = comp["new"]
            reference = comp["reference"]
            pearson = sp.stats.pearsonr(reference.astype(float), new.astype(float))
            indicators.loc[plant]["pearson_h"] = pearson[0]
            indicators.loc[plant]["mbe_h"] = (np.sum(new-reference))/np.sum(reference.size)
            observations = comp.size/comp.columns.size
            indicators.loc[plant]["observations"] = observations
            indicators.loc[plant]["rmse_h"] = np.sqrt((np.sum((new-reference)**2))/(observations))
            #indicators.loc[plant]["rmse_r_h"] = np.sqrt((np.sum((new-reference)**2))/observations)/(np.sum(reference/observations))
            new_avg_day = round(new.resample('D').mean(),4).dropna().copy()
            days = new_avg_day.size
            indicators.loc[plant]["days"] = days
            reference_avg_day = round(reference.astype(float).resample('D').mean(),4).dropna().copy()
            pearson_days = sp.stats.pearsonr(reference_avg_day.astype(float), new_avg_day.astype(float))
            indicators.loc[plant]["pearson_d"] = pearson_days[0]
            indicators.loc[plant]["mbe_d"] = (np.sum(new_avg_day-reference_avg_day))/np.sum(reference_avg_day.size)
            indicators.loc[plant]["rmse_d"] = np.sqrt((np.sum((new_avg_day-reference_avg_day)**2))/(np.sum(reference_avg_day.size)))
            new_avg_month = round(new.resample('M').mean(),4).dropna().copy()
            months = new_avg_month.size 
            if months > 1:
                #print(plant)
                reference_avg_month = round(reference.astype(float).resample('M').mean(),4).dropna().copy()
                pearson_months = sp.stats.pearsonr(reference_avg_month.astype(float), new_avg_month.astype(float))
                indicators.loc[plant]["pearson_m"] = pearson_months[0]
                indicators.loc[plant]["mbe_m"] = (np.sum(new_avg_month-reference_avg_month))/np.sum(reference_avg_month.size)
                indicators.loc[plant]["rmse_m"] = np.sqrt((np.sum((new_avg_month-reference_avg_month)**2))/(np.sum(reference_avg_month.size)))
                indicators.loc[plant]["months"] = months
    #drop all installations where no meassured data is available
    indicators = indicators.dropna().copy()
    #drop all installations where less than 12 months of data are available
    minimum_months = 12
    indicators = indicators.loc[indicators["months"] > minimum_months].drop(["observations",
                                                                              "days",
                                                                              "months"], axis=1).copy()
    indicators.to_csv(name_file_to_store)
    return indicators

In [13]:
#calculate the indicators between raw meassured data and all ERA5 and MERRA2 
#based estimations for tracking and non-tracking systems
indicators_era5l_optimal = validate_pv_output_all(reference,
                                                  era5l_optimal,
                                                  Path('intermediate_results/indicators_era5l_optimal.csv'))
indicators_era5l_tracking = validate_pv_output_all(reference,
                                                  era5l_tracking,
                                                  Path('intermediate_results/indicators_era5l_tracking.csv'))
indicators_merra2_optimal = validate_pv_output_all(reference,
                                                  merra2_optimal,
                                                  Path('intermediate_results/indicators_merra2_optimal.csv'))
indicators_merra2_tracking = validate_pv_output_all(reference,
                                                  merra2_tracking,
                                                  Path('intermediate_results/indicators_merra2_tracking.csv'))

In [14]:
#calculate the indicators between deseasonalized meassured data and all deseasonalized ERA5 and MERRA2 
#based estimations for tracking and non-tracking systems
indicators_era5l_optimal_des_rad = \
validate_pv_output_all(reference_optimal_des_rad,
                       era5l_optimal_des_rad,
                       Path('intermediate_results/indicators_era5l_optimal_deseason_rad.csv'))
indicators_era5l_tracking_des_rad =\
validate_pv_output_all(reference_tracking_des_rad,
                       era5l_tracking_des_rad,
                       Path('intermediate_results/indicators_era5l_tracking_deseason_rad.csv'))
indicators_merra2_optimal_des_rad = \
validate_pv_output_all(reference_optimal_des_rad,
                       merra2_optimal_des_rad,
                       Path('intermediate_results/indicators_merra2_optimal_deseason_rad.csv'))
indicators_merra2_tracking_des_rad = \
validate_pv_output_all(reference_tracking_des_rad,
                       merra2_tracking_des_rad,
                       Path('intermediate_results/indicators_merra2_tracking_deseason_rad.csv'))

In [15]:
indicators_era5l_optimal_cumulated = \
validate_pv_output_all(reference_non_tracking_cumulated,
                       era5l_optimal_cumulated,
                       Path('intermediate_results/indicators_era5l_optimal_cumulated.csv'))
indicators_era5l_tracking_cumulated =\
validate_pv_output_all(reference_tracking_cumulated,
                       era5l_tracking_cumulated,
                       Path('intermediate_results/indicators_era5l_tracking_cumulated.csv'))
indicators_merra2_optimal_cumulated = \
validate_pv_output_all(reference_non_tracking_cumulated,
                       merra2_optimal_cumulated,
                       Path('intermediate_results/indicators_merra2_optimal_cumulated.csv'))
indicators_merra2_tracking_cumulated = \
validate_pv_output_all(reference_tracking_cumulated,
                       merra2_tracking_cumulated,
                       Path('intermediate_results/indicators_merra2_tracking_cumulated.csv'))

In [16]:
indicators_era5l_optimal_des_cumulated = \
validate_pv_output_all(reference_non_tracking_des_cumulated,
                       era5l_optimal_des_rad_cumulated,
                       Path('intermediate_results/indicators_era5l_optimal_deseason_cumulated.csv'))
indicators_era5l_tracking_des_cumulated =\
validate_pv_output_all(reference_tracking_des_cumulated,
                       era5l_tracking_des_rad_cumulated,
                       Path('intermediate_results/indicators_era5l_tracking_deseason_cumulated.csv'))
indicators_merra2_optimal_des_cumulated = \
validate_pv_output_all(reference_non_tracking_des_cumulated,
                       merra2_optimal_des_rad_cumulated,
                       Path('intermediate_results/indicators_merra2_optimal_deseason_cumulated.csv'))
indicators_merra2_tracking_des_cumulated = \
validate_pv_output_all(reference_tracking_des_cumulated,
                       merra2_tracking_des_rad_cumulated,
                       Path('intermediate_results/indicators_merra2_tracking_deseason_cumulated.csv'))

## Visualize some results 

In [20]:
#import classification of installations based on automatic procedure
installations_selection = pd.read_csv(Path('intermediate_results/installation_classification.csv'),
                                      index_col=1) 

In [21]:
installations_selection

,Unnamed: 0,Type
Location,,
BELLAVISTA,1,erroneous
CARRERA PINTO ETAPA I,2,erroneous
CHANARES,3,erroneous
CONEJO SOLAR,4,erroneous
FV BOLERO,5,erroneous
LALACKAMA,6,erroneous
LALACKAMA 2,7,erroneous
LOS PUQUIOS,8,erroneous
LUZ DEL NORTE,9,erroneous


In [22]:
#installations_selection

In [23]:
results_era5l_optimal_all = pd.read_csv(Path('intermediate_results/indicators_era5l_optimal.csv'),
                                    index_col=0)
results_era5l_tracking_all = pd.read_csv(Path('intermediate_results/indicators_era5l_tracking.csv'),
                                    index_col=0)
results_era5l_tracking_tracking = \
results_era5l_tracking_all.loc[installations_selection.index[installations_selection.Type == 'tracking']]

results_era5l_optimal_fixed = \
results_era5l_optimal_all.loc[installations_selection.index[installations_selection.Type == 'non-tracking']]

In [91]:
#indicators_era5l_optimal.loc[installations_selection.index[installations_selection.Type == 'non-tracking']].mean() 
#indicators_era5l_optimal.loc[installations_selection.index[installations_selection.Type == 'non-tracking']].median() 
indicators_era5l_optimal.loc[installations_selection.index[installations_selection.Type == 'non-tracking']].max()
#indicators_era5l_optimal.loc[installations_selection.index[installations_selection.Type == 'non-tracking']].min()

pearson_h    0.974875
mbe_h        0.078497
rmse_h       0.197689
pearson_d    0.864539
mbe_d        0.079230
rmse_d       0.130563
pearson_m    0.901499
mbe_m        0.086090
rmse_m       0.107984
dtype: float64

In [85]:
#indicators_merra2_optimal.loc[installations_selection.index[installations_selection.Type == 'non-tracking']].mean()
#indicators_merra2_optimal.loc[installations_selection.index[installations_selection.Type == 'non-tracking']].median()
indicators_merra2_optimal.loc[installations_selection.index[installations_selection.Type == 'non-tracking']].max()
#indicators_merra2_optimal.loc[installations_selection.index[installations_selection.Type == 'non-tracking']].min()

,pearson_h,mbe_h,rmse_h,pearson_d,mbe_d,rmse_d,pearson_m,mbe_m,rmse_m
Location,,,,,,,,,
LUNA DEL NORTE,0.92244,0.0222364,0.124361,0.699056,0.0265536,0.0852982,0.742616,0.027365,0.0501024
PFV LOS LOROS,0.838945,0.0768183,0.200592,0.848384,0.0795276,0.132793,0.893414,0.0867161,0.109496
SOL DEL NORTE,0.920773,0.0192229,0.126097,0.684318,0.0234443,0.0863132,0.746793,0.0209769,0.0459174
SOLAR EL AGUILA I,0.95505,-0.0111257,0.0978115,0.657964,-0.0101205,0.0617062,0.804449,-0.0106692,0.0269677
SOLAR HORMIGA,0.913545,0.0507114,0.144562,0.577676,0.0585232,0.112129,0.254065,0.0571467,0.0803346
SOLAR SANTA CECILIA,0.902548,0.0168137,0.136549,0.660341,0.0194325,0.0872915,0.846335,0.0180123,0.0394792
SPS LA HUAYCA,0.964514,0.0145849,0.0884594,0.752323,0.0159277,0.055801,0.887042,0.0154194,0.0239518
LAS MOLLACAS,0.850957,0.0500647,0.184216,0.581415,0.0547677,0.112751,0.68273,0.0535833,0.0671883
SOLAR PSF LOMAS COLORADAS,0.926878,0.0176541,0.120196,0.772756,0.0213064,0.080268,0.74744,0.0212926,0.0433131


In [92]:
#indicators_era5l_tracking.loc[installations_selection.index[installations_selection.Type == 'tracking']].mean() 
indicators_era5l_tracking.loc[installations_selection.index[installations_selection.Type == 'tracking']]  
indicators_era5l_tracking.loc[installations_selection.index[installations_selection.Type == 'tracking']].max()  

pearson_h    0.947101
mbe_h        0.098799
rmse_h       0.225105
pearson_d    0.908241
mbe_d        0.099155
rmse_d       0.148073
pearson_m    0.987194
mbe_m        0.099861
rmse_m       0.108824
dtype: float64

In [78]:
#indicators_merra2_tracking.loc[installations_selection.index[installations_selection.Type == 'tracking']].mean()
indicators_merra2_tracking.loc[installations_selection.index[installations_selection.Type == 'tracking']].median()
#indicators_merra2_tracking.loc[installations_selection.index[installations_selection.Type == 'tracking']].max() 

pearson_h    0.846170
mbe_h        0.064605
rmse_h       0.195522
pearson_d    0.789867
mbe_d        0.066247
rmse_d       0.124345
pearson_m    0.948932
mbe_m        0.067141
rmse_m       0.087996
dtype: float64

In [88]:
#indicators_era5l_optimal_des_rad.loc[installations_selection.index[installations_selection.Type == 'non-tracking']].mean()
indicators_era5l_optimal_des_rad.loc[installations_selection.index[installations_selection.Type == 'non-tracking']]
#indicators_era5l_optimal_des_rad.loc[installations_selection.index[installations_selection.Type == 'non-tracking']].max() 

,pearson_h,mbe_h,rmse_h,pearson_d,mbe_d,rmse_d,pearson_m,mbe_m,rmse_m
Location,,,,,,,,,
LUNA DEL NORTE,0.745135,-0.0157287,0.111669,0.87845,-0.0122744,0.0692896,0.962819,-0.0114025,0.0393219
PFV LOS LOROS,0.479917,0.0784967,0.197689,0.670717,0.0792294,0.130564,0.803522,0.0861032,0.107999
SOL DEL NORTE,0.742742,-0.0191446,0.113457,0.877364,-0.0157127,0.0707234,0.96996,-0.0177103,0.0414198
SOLAR EL AGUILA I,0.641936,-0.0209328,0.108684,0.75633,-0.0189387,0.074939,0.846958,-0.0200462,0.0546152
SOLAR HORMIGA,0.72293,0.0309299,0.118144,0.833834,0.0367737,0.084734,0.938672,0.0361733,0.0606286
SOLAR SANTA CECILIA,0.663443,0.0147659,0.129772,0.831333,0.0164032,0.0763647,0.954828,0.0150491,0.0343047
SPS LA HUAYCA,0.814283,0.00509134,0.073109,0.917336,0.00628591,0.0418359,0.980263,0.00568333,0.0196867
LAS MOLLACAS,0.50025,0.0508386,0.172046,0.72642,0.0544722,0.105335,0.958431,0.0537,0.0627523
SOLAR PSF LOMAS COLORADAS,0.784989,0.0255085,0.106938,0.90104,0.0276793,0.0674733,0.959565,0.0281944,0.041678


In [81]:
#indicators_merra2_optimal_des_rad.loc[installations_selection.index[installations_selection.Type == 'non-tracking']].mean()
indicators_merra2_optimal_des_rad.loc[installations_selection.index[installations_selection.Type == 'non-tracking']].median()
#indicators_merra2_optimal_des_rad.loc[installations_selection.index[installations_selection.Type == 'non-tracking']].max()

pearson_h    0.678456
mbe_h        0.019223
rmse_h       0.126097
pearson_d    0.806261
mbe_d        0.023446
rmse_d       0.086309
pearson_m    0.953959
mbe_m        0.021300
rmse_m       0.045925
dtype: float64

In [89]:
indicators_era5l_tracking_des_rad.loc[installations_selection.index[installations_selection.Type == 'tracking']].mean()
indicators_era5l_tracking_des_rad.loc[installations_selection.index[installations_selection.Type == 'tracking']]
#indicators_era5l_tracking_des_rad.loc[installations_selection.index[installations_selection.Type == 'tracking']].max() 

,pearson_h,mbe_h,rmse_h,pearson_d,mbe_d,rmse_d,pearson_m,mbe_m,rmse_m
Location,,,,,,,,,
ANDES SOLAR,0.38504,0.0823174,0.160711,0.536038,0.0816518,0.121112,0.49657,0.0831086,0.101034
EL DIVISADERO,0.612328,-0.0173542,0.173967,0.748499,-0.0174507,0.0985728,0.810668,-0.0203571,0.0441362
LA CHAPEANA,0.52782,0.0987986,0.212927,0.707217,0.0991589,0.134092,0.658724,0.0998694,0.108831
LA QUINTA SOLAR,0.661222,0.0570979,0.171014,0.850722,0.058918,0.0955721,0.83603,0.057825,0.0609785
LAS TURCAS,0.542785,0.0878578,0.225105,0.69173,0.0879686,0.148077,0.416639,0.0840333,0.101184
MARIA ELENA FV,0.542003,0.0301401,0.110102,0.78395,0.0306448,0.0574688,0.897041,0.030575,0.0362301
PARQUE SOLAR PAMPA CAMARONES,0.419745,0.0293769,0.172811,0.513941,0.0324209,0.117605,0.53029,0.0309812,0.0827997
PUERTO SECO SOLAR,0.48214,-0.0516411,0.12553,0.695144,-0.0510325,0.0817999,0.820737,-0.0520056,0.0595354
SANTA JULIA,0.692553,0.0592096,0.172221,0.843489,0.0578733,0.10049,0.793683,0.0570188,0.0658655


In [73]:
indicators_merra2_tracking_des_rad.loc[installations_selection.index[installations_selection.Type == 'tracking']].mean()
indicators_merra2_tracking_des_rad.loc[installations_selection.index[installations_selection.Type == 'tracking']].max() 

pearson_h    0.560391
mbe_h        0.114208
rmse_h       0.240134
pearson_d    0.714592
mbe_d        0.114939
rmse_d       0.153763
pearson_m    0.899158
mbe_m        0.114019
rmse_m       0.122780
dtype: float64

In [ ]:
'''
selected_start = '2018-06-01'
selected_end = '2018-06-03'
plant = 'SOLAR ANTAY'
reference.loc[selected_start:selected_end,plant].plot(figsize=(20,6))
era5l_optimal.loc[selected_start:selected_end,plant].plot(figsize=(20,6))
era5l_tracking.loc[selected_start:selected_end,plant].plot(figsize=(20,6))
'''

In [ ]:

selected_start = '2018-12-01'
selected_end = '2018-12-03'
reference.loc[selected_start:selected_end,plant].plot(figsize=(20,6))
era5l_optimal.loc[selected_start:selected_end,plant].plot(figsize=(20,6))
era5l_tracking.loc[selected_start:selected_end,plant].plot(figsize=(20,6))


In [ ]:
'''
selected_start = '2018'
selected_end = '2018'
plant = 'SOLAR ANTAY'
era5l_optimal.loc[selected_start:selected_end,plant].plot(figsize=(20,6))
reference.loc[selected_start:selected_end,plant].plot(figsize=(20,6))
#era5l_tracking.loc[selected_start:selected_end,plant].plot(figsize=(20,6))
'''

In [ ]:
selected_start = '2017'
selected_end = '2018'
plant = 'SOLAR HORMIGA'
era5l_optimal.loc[selected_start:selected_end,plant].resample('M').mean().plot(figsize=(20,6))
reference.loc[selected_start:selected_end,plant].resample('M').mean().plot(figsize=(20,6))
#era5l_tracking.loc[selected_start:selected_end,plant].plot(figsize=(20,6))